In [0]:
%sql
select * from delta.`/mnt/bronze/multiplexbronzetable` where source='orders';

In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *

# Define the schema inline
orders_schema = StructType([
    StructField("order_id", StringType()),
    StructField("customer_id", StringType()),
    StructField("order_date", DateType()),
    StructField("status", StringType()),
    StructField("payment_method", StringType()),
    StructField("shipping_method", StringType()),
    StructField("shipping_address", StringType()),
    StructField("total_amount", DoubleType()),
    StructField("items", ArrayType(StructType([
        StructField("product_id", StringType()),
        StructField("quantity", IntegerType()),
        StructField("price", DoubleType()),
        StructField("discount", DoubleType()),
        StructField("subtotal", DoubleType())
    ])))
])

raw_orders = spark.readStream.format("delta").load("/mnt/bronze/multiplexbronzetable") \
    .filter("source = 'orders'")

parsed_orders = raw_orders.withColumn(
    "parsed", from_json(col("raw_payload").cast("string"), orders_schema)
).select("parsed.*", "ingesttimestamp")

deduped_orders = parsed_orders \
    .withWatermark("ingesttimestamp", "10 minutes") \
    .dropDuplicates(["order_id", "customer_id"])



In [0]:
display(deduped_orders)

In [0]:
%sql
create table if not exists orders_silver(
  order_id string,customer_id string,order_date date,status string,payment_method string,shipping_method string,shipping_address string,total_amount double, items array<struct<product_id:string,quantity:int,price:double,discount:double,subtotal:double>>,
  ingesttimestamp timestamp
) 

Location "/mnt/silver/orders_silver";

In [0]:
%sql
drop table orders_silver

In [0]:
deduped_orders.writeStream\
    .format("delta")\
        .option("checkpointLocation","/mnt/checkpoint/silver/orders2")\
            .outputMode("append")\
                .option("mergeSchema","true")\
            .start("/mnt/silver/orders_silver")        
            

In [0]:
%sql
select * from orders_silver